# Data Acquisition

from time import sleep
from bs4 import BeautifulSoup
import time
import requests
import json
import unicodedata

In [1]:
#import the libraries needed for web scraping
from time import sleep
from bs4 import BeautifulSoup
import time
import requests
import json
import unicodedata

### Using the BeautifulSuop scarper api to scrape the data from wine.com, we will scrape the landing page and iterate through the win varieties and scrape results page and product page an placing it into a dictionary for futher analysis 

In [ ]:
#Creating the scraper 
class Scraper:
#initializing the web scaper for extracting data from wine.com
    def __init__(self, pullQuantity = 10000):
        self.session = requests.Session()
        self.startTime = time.time()
        self.pullQuantity = pullQuantity
        self.staticURL = 'https://www.wine.com'
        self.siteMapDirectory = '../../data/wine-com/'
        self.filePath = '../../data/wine-com/raw/{}.txt'.format(self.startTime)
        self.headerData = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

       #determine if sitemap exists, initialize if not
        try:
            with open(self.siteMapDirectory + '/site-map.json', 'r') as fileObj:
                self.siteMap = json.load(fileObj)
        except:
            self.siteMap = {}
        
        #generate session file - validated
        with open(self.filePath,'w') as fileObj:
            fileObj.write('product_url|product_name|product_variety|product_origin|product_family|user_avg_rating|user_rating_count|product_description|product_reviews\n')  
        fileObj.close()
            
    def scrape(self):
        #determine landing page map exists
        if len(self.siteMap) == 0:
            self.scrapeLandingPage()
            self.parseLandingPage()
        #iterate through wine varieties
        sleep(2)
        for key, value in self.siteMap['landingPageLinks'].items():
            self.searchVarietal = key
            self.searchURL = value
            self.scrapeParseResultsPage()
            #write sitemap to JSON file
            with open(self.siteMapDirectory + '/site-map.json', 'w') as fileObj:
                json.dump(self.siteMap, fileObj)
            fileObj.close()

    # validated
    def scrapeLandingPage(self):
        landingPageResponse = self.session.get(self.staticURL, headers = self.headerData)
        self.landingPageSoup = BeautifulSoup(landingPageResponse.content, "html.parser")
    
    #validated
    def parseLandingPage(self):
        self.siteMap['landingPageLinks'] = dict()
        landingPageVarietalsLevel0 = self.landingPageSoup.find('section', attrs = {'class': 'mainNav_section mainNav_section-varietal mainNavList_item-level0'})
        for row in landingPageVarietalsLevel0.find_all('li', attrs = {'class': 'mainNavList_item mainNavList_item-level2'}):
            a_tag = row.find('a', attrs = {'class': 'mainNavList_itemLink'})
            self.varietalName = a_tag.text
            self.varietalLink = a_tag.get('href')
            self.siteMap['landingPageLinks'][self.varietalName] = self.staticURL + self.varietalLink
        #for some reason boxed sets and glassware links are present in varietals tab
        del self.siteMap['landingPageLinks']['Wine Tasting Sets']
        del self.siteMap['landingPageLinks']['Glassware & Accessories']
    
    def scrapeParseResultsPage(self):
        if self.searchVarietal not in self.siteMap:
            self.siteMap[self.searchVarietal] = dict()
        
        self.pullCount = 0
        while self.pullCount < self.pullQuantity and self.searchURL is not None:
            print(self.searchURL)
            resultsPageResponse = self.session.get(self.searchURL, headers = self.headerData)
            self.resultsPageSoup = BeautifulSoup(resultsPageResponse.content, "html.parser")
            resultsContainer = self.resultsPageSoup.find('ul', attrs = {'class':'listGridLayout_list'})
            for row in resultsContainer.find_all('div', attrs = {'class': 'listGridItemInfo'}):
                productShortLink = row.a['href']
                self.productURL = self.staticURL + productShortLink
                if self.productURL not in self.siteMap[self.searchVarietal]:
                    self.scrapeProductPage()
                    self.parseProductPage()
                else:
                    pass
            if self.pullCount < self.pullQuantity:
                try:
                    paginationContainer = self.resultsPageSoup.find('div', attrs = {'class':'nextPagePagination'})
                    self.searchURL = paginationContainer.a['href']
                except Exception:
                    self.searchURL = None
            
    #validated
    def scrapeProductPage(self):
        if self.productURL not in self.siteMap[self.searchVarietal]:
            self.siteMap[self.searchVarietal][self.productURL] = dict()
        
        try:
            productPageResponse = self.session.get(self.productURL, headers = self.headerData)
            self.productPageSoup = BeautifulSoup(productPageResponse.content, "html.parser")
            
            self.siteMap[self.searchVarietal][self.productURL]['scrapeStatus'] = 'success'
        except Exception:
            self.siteMap[self.searchVarietal][self.productURL]['scrapeStatus'] = 'fail'
            
    #validated
    def parseProductPage(self):
        try:
            #generate dictionary for parsed fields
            self.prodData = dict()
            
            #product info
            prodInfo = self.productPageSoup.find('div', attrs = {'class':'pipInfo'})
            self.prodData['Product_Name'] = prodInfo.find('h1', attrs = {'class':'pipName'}).text
            self.prodData['Product_Variety'] = prodInfo.find('span', attrs = {'class':'prodItemInfo_varietal'}).text
            self.prodData['Product_Origin'] = prodInfo.find('span', attrs = {'class':'prodItemInfo_originText'}).text
            
            #product attributes
            self.prodData['Product_Family'] = self.searchVarietal
            
            try:
                #average user product ratings
                self.prodData['User_Avg_Rating'] = self.productPageSoup.find('span', attrs = {'class':'averageRating_average'}).text
            except Exception:
                self.prodData['User_Avg_Rating'] = 'nan'
                
            try:
                #product ratings count
                self.prodData['User_Rating_Count'] = self.productPageSoup.find('span', attrs = {'class':'averageRating_number'}).text
            except Exception:
                self.prodData['User_Rating_Count'] = 'nan'
                
            try:
                #winemaker notes
                prodNotes = self.productPageSoup.find('div', attrs = {'class':'pipWineNotes_copy viewMoreModule js-expanded'})
                wineMakerNotes = prodNotes.find('div', attrs = {'class': 'viewMoreModule_text'}).text
                wineMakerNotes = wineMakerNotes.replace('\n', ' ')
                wineMakerNotes = unicodedata.normalize('NFKD', wineMakerNotes)
                self.prodData['Winemaker_Description'] = wineMakerNotes
            except Exception:
                self.prodData['Winemaker_Description'] = 'nan'
            
            try:
                #professional reviews
                prodProfessionalReviews = self.productPageSoup.find('div', attrs = {'class': 'viewMoreModule_text viewMoreModule-reviews'})
                self.prodData['Critical_Reviews'] = []
                for row in prodProfessionalReviews.find_all('div', attrs = {'class': 'pipProfessionalReviews_list'}):
                    reviewer_name = row.find('div', attrs = {'class': 'pipProfessionalReviews_authorName'}).text
                    reviewer_rating = row.find('span', attrs = {'class': 'wineRatings_rating'}).text
                    reviewer_text = row.find('div', attrs = {'class': 'pipSecContent_copy'}).text
                    reviewer_text = reviewer_text.replace('\n', ' ')
                    reviewer_text = unicodedata.normalize('NFKD', reviewer_text)                             
                    self.prodData['Critical_Reviews'].append(f'({reviewer_name}; {reviewer_rating}; {reviewer_text})')
            except Exception:
                print('review parse failed')
                self.prodData['Critical_Reviews'] = 'nan'

            #write data to disk
            self.writeProductData()
            self.siteMap[self.searchVarietal][self.productURL]['parseStatus'] = 'success'
        except Exception:
            self.siteMap[self.searchVarietal][self.productURL]['parseStatus'] = 'fail'
            print('{} parse failed'.format(self.productURL))

    #validated
    def writeProductData(self):
        try:
            with open(self.filePath, 'a') as file:
                file.write('{}|{}|{}|{}|{}|{}|{}|{}|{}\n'.format(self.productURL,
                                                                 self.prodData['Product_Name'],
                                                                 self.prodData['Product_Variety'],
                                                                 self.prodData['Product_Origin'],
                                                                 self.prodData['Product_Family'],
                                                                 self.prodData['User_Avg_Rating'],
                                                                 self.prodData['User_Rating_Count'],
                                                                 self.prodData['Winemaker_Description'],
                                                                 self.prodData['Critical_Reviews']))
            file.close()
            self.pullCount += 1
            self.siteMap[self.searchVarietal][self.productURL]['writeStatus'] = 'success'
        except Exception:
            self.siteMap[self.searchVarietal][self.productURL]['writeStatus'] = 'fail'


In [ ]:
wine_com_scraper = Scraper()
wine_com_scraper.scrape()

# Preprocessing - Wine.com

### Summary
The code contained within this file aims to reconcile and process data scraped from Wine.com.

This implementation observes the following considerations:
1. Given that scrapes can and will fail, this script assumes that all files present in the raw folder are to be processed and combined into a master record. Redundant records and processing are prevented through the use of a site map that tracks successfully parsed and written pages.
2. Ideally, this data would best be stored in a database with a mininum of two tables, product info and critical reviews, however as this analysis is predominantly interested in review data, a SQL style left join has been deemed adequate for this analysis despite the effect on the overall file size. This is a case of not letting perfect get in the way of good enough or having an "agile" mindset.
3. The preprocess_text.py file contains the preprocess_text Class to be used in pipeline when addressing novel data.

In [9]:
#imports
import numpy as np
import pandas as pd
import os
import re
import string
import time
from sklearn.base import TransformerMixin
from nltk.corpus import stopwords

In [2]:
#set directory locations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
raw_folder = parent_directory + '/data/wine-com/raw/'
processed_folder = parent_directory + '/data/wine-com/processed/'

### Load Data

In [5]:
#instantiate data structure to hold initial raw data/critical critical reviews
data = dict()
review_data = dict()
#iterate through wine.com raw data folder
for filename in os.listdir(raw_folder):
    #determine if file has already been processed - if true pass
    if filename not in os.listdir(processed_folder):
        print(f'{filename} loading')
        data[filename] = dict()
        review_data[filename] = dict()
        #open file readlines to raw dictionary
        with open(raw_folder + filename, newline='\r\n') as file:
            header = next(file)
            data[filename]['lines'] = file.readlines()
        #create list to hold parsed records
        data[filename]['data'] = []
        review_data[filename]['data'] = []
        #iterate through records to parse raw data
        for line in data[filename]['lines']:
            values = line.strip().split('|')
            #while redundancy is built into the scraper to ensure that the correct format is achieved,
            #this prevents rogue html from causing a bad data load
            if len(values) == 9:
                row = {
                    'product_url': values[0],
                    'product_name': values[1],
                    'product_variety': values[2],
                    'product_origin': values[3],
                    'product_family': values[4],
                    'user_avg_rating': values[5],
                    'user_rating_count': values[6],
                    'winemaker_description': values[7]
                    #'critical_reviews' is parsed loaded below
                }
                #write to list to be loaded to dataframe
                data[filename]['data'].append(row)
                
                #parse critical reviews - should be loaded as list
                reviews = values[8].split(';')
                for review in reviews:
                    try:
                        reviewer_name, reviewer_rating, reviewer_text = review.split(',')
                        review = {
                        'product_url': values[0],
                        'reviewer_name': reviewer_name,
                        'reviewer_rating': reviewer_rating,
                        'reviewer_text': reviewer_text
                        }
                        
                        review_data[filename]['data'].append(review)
                    except Exception:
                        pass      
                    
        # Convert the list of dictionaries to a Pandas DataFrame
        data[filename]['unmerged_df'] = pd.DataFrame(data[filename]['data'])
        review_data[filename]['df'] = pd.DataFrame(review_data[filename]['data'])
        
        #perform sql-style left join of review data onto main df
        data[filename]['merged_df'] = pd.merge(data[filename]['unmerged_df'], 
                                               review_data[filename]['df'], 
                                               on='product_url', 
                                               how='left')

1677386048.5362737.txt loading
1677417882.7459548.txt loading


We created two .txt file one for raw data and another for ciritcal review  

### Data Preprocessing

Using the transformerMixin to preprocess text data by removing stopwords, punctuation, and converting text to lowercase.

In [19]:
# a text preprocessor class is created to manage preprocessing of text fields
# inheriting from TransformerMixin of Sklearn, this should allow for tying into large sklearn pipeline
class preprocess_text(TransformerMixin):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
    
    def transform(self, X):
        if isinstance(X, pd.Series):
            #standardize type
            X = X.astype(str)
            # lower text
            X = X.str.lower()
            # remove punctuation
            X = X.str.replace('\W+', ' ', regex = True)
            # tokenize text into individual words
            X = X.str.split()
            # remove stopwords
            X = X.apply(lambda x: [word for word in x if word not in (self.stop_words)])
            # join words
            X = X.apply(lambda x: ' '.join(x))
            return X
        elif isinstance(X, pd.DataFrame):
            colnames = X.columns
            for col in colnames:
                #standardize type
                X[col] = X[col].astype(str)
                # convert text to lowercase
                X[col] = X[col].str.lower()
                # remove punctuation
                X[col] = X[col].str.replace('\W+', ' ', regex = True)
                # tokenize text into individual words
                X[col] = X[col].str.split()
                # remove stopwords
                X[col] = X[col].apply(lambda x: [word for word in x if word not in (self.stop_words)])
                # join words
                X[col] = X[col].apply(lambda x: ' '.join(x))
            return X
        else:
            return X
    
    def fit(self, X, y=None):
        return self

### Clean & Write Data

applying the processor to the winemaker_description and reviewer_text and saving it to a .txt 

In [20]:
processor = preprocess_text()

output_path = processed_folder + f'{time.time()}.txt'

for filename in data:
    data[filename]['merged_df']['winemaker_description'] = processor.transform(data[filename]['merged_df']['winemaker_description'])
    data[filename]['merged_df']['reviewer_text'] = processor.transform(data[filename]['merged_df']['reviewer_text'])
    data[filename]['merged_df'].to_csv(output_path,
                                       mode='a',
                                       header = not os.path.exists(output_path),
                                       sep = '|',
                                       line_terminator = '\r\n',
                                       index=False)